In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
!pip install tensorflow==2.15.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [2]:
from google.colab import drive
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Check GPU availability
print("GPU Available: ", tf.test.gpu_device_name())

# Mount Google Drive
drive.mount('/content/gdrive/')

GPU Available:  /device:GPU:0
Mounted at /content/gdrive/


In [5]:
# Define paths to the dataset and extract it
dataset_zip_path = '/content/gdrive/My Drive/PlaasHelper/Crops.zip'  # Adjust the path if needed
extracted_dataset_path = '/content/dataset/Crops/Crops/Crops'

In [6]:
# Extract the dataset
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dataset_path)

root_path = extracted_dataset_path

# Define image size and batch size
target_size = (150, 150)  # You can adjust this size as needed
batch_size = 32

# Define data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(root_path, 'train'),
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(root_path, 'valid'),
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define the model using VGG16 as the base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])



Found 14629 images belonging to 16 classes.
Found 4628 images belonging to 16 classes.
58889256/58889256 [==============================] - 4s 0us/step


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/10
170/457 [==========>...................] - ETA: 3:37 - loss: 0.7756 - accuracy: 0.7375

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


457/457 [==============================] - 365s 755ms/step - loss: 0.5134 - accuracy: 0.8276 - val_loss: 3.1167 - val_accuracy: 0.4944
Epoch 2/10
457/457 [==============================] - 314s 687ms/step - loss: 0.2211 - accuracy: 0.9224 - val_loss: 2.8033 - val_accuracy: 0.5623
Epoch 3/10
457/457 [==============================] - 316s 690ms/step - loss: 0.1577 - accuracy: 0.9455 - val_loss: 3.1312 - val_accuracy: 0.5575
Epoch 4/10
457/457 [==============================] - 316s 690ms/step - loss: 0.1305 - accuracy: 0.9546 - val_loss: 2.2735 - val_accuracy: 0.5538
Epoch 5/10
457/457 [==============================] - 355s 776ms/step - loss: 0.1014 - accuracy: 0.9649 - val_loss: 3.6678 - val_accuracy: 0.5612
Epoch 6/10
457/457 [==============================] - 326s 711ms/step - loss: 0.0887 - accuracy: 0.9703 - val_loss: 3.0298 - val_accuracy: 0.5840
Epoch 7/10
457/457 [==============================] - 326s 713ms/step - loss: 0.0717 - accuracy: 0.9760 - val_loss: 3.3469 - val_accura

In [8]:
# Save the trained model to Google Drive
model_save_path = '/content/gdrive/My Drive/PlaasHelper/crop_classification_modell.h5'
model.save(model_save_path)

print(f"Training completed and model saved at {model_save_path}.")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training completed and model saved at /content/gdrive/My Drive/PlaasHelper/crop_classification_modell.h5.


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Example setup
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/dataset/Crops/Crops/Crops/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

class_labels = train_generator.class_indices
print(class_labels)


Found 14629 images belonging to 16 classes.
{'Apple Blotch': 0, 'Apple Healthy Apple': 1, 'Apple Rotten Apple': 2, 'Apple Scab': 3, 'Corn (Maize) Ear Rot': 4, 'Corn (Maize) Fall Armyworm': 5, 'Corn (Maize) Healthy Corn (Maize)': 6, 'Corn (Maize) Stem Borer': 7, 'Grapes Healthy Grapes': 8, 'Potato Healthy Potato': 9, 'Potato Rotten Potato': 10, 'Strawberry Healthy Strawberry or Pickable Strawberry': 11, 'Strawberry Rotten Strawberry': 12, 'Strawberry Unpickable Strawberry': 13, 'Tomato Ripe Tomato or Healthy Tomato': 14, 'Tomato Rotten Tomato': 15}


In [ ]:
from tensorflow.keras.models import load_model

# Load your model
model = load_model('/content/gdrive/My Drive/PlaasHelper/crop_classification_modell.h5')

# Inspect the model architecture
model.summary()

# Check the output layer
output_layer = model.layers[-1]
print(output_layer.get_config())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               4194816   
                                                                 
 dense_3 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 18913608 (72.15 MB)
Trainable params: 18913608 (72.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
{'name': 'dense_3', 'trainable': True, 'dtype': 'float32', 'units': 8, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'module